In [ ]:
using Revise, ReachabilityAnalysis, BenchmarkTools

const RA = ReachabilityAnalysis
RA.AbstractLazyReachSet
using Plots

┌ Info: Precompiling ReachabilityAnalysis [1e97bd63-91d1-579d-8e8d-501d2b57c93f]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


In [ ]:
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/hybrid/embrake.jl")

# Flowpipe no pv

In [ ]:
prob = embrake_no_pv(ζ=1e-5);

In [ ]:
typeof(prob)

In [ ]:
@time sol = solve(prob, alg=GLGM06(δ=1e-6), max_jumps=1000);

In [ ]:
#@time sol = solve(prob, alg=GLGM06(δ=1e-8), max_jumps=1000, force_static=false);

## Requirements

1) Variable $x(t)$ is such that $\Vert x(t) - x_0 \Vert \leq \varepsilon$ for all $t \geq t_1$. We take $t_1 = 23ms$ and $x_0 = 0.05$.

In [ ]:
x0 = 0.05
ε = 2e-3

2) The speed does is not bigger than a certain threshold  value once the disc makes contact: $\vert v(t) \vert \leq v_{max}$ for all $t \geq t_1$. We take $t_1 = 23ms$ and $v_{max} = 2mm/s$.

### Requirement 1

In [ ]:
Dx⁺ = [0, 1.0, 0, 0]
Dx⁻ = [0, -1.0, 0, 0]

In [ ]:
# prueba
using LazySets.Approximations

R = ReachSet(rand(Zonotope, dim=2), 0 .. 1)

dirs_box = [Vector(v) for v in collect(BoxDirections(2))]
Rbox = overapproximate(R, dirs_box)

dirs_oct = [Vector(v) for v in collect(OctDirections(2))]
Roct = overapproximate(R, dirs_oct);

dirs_pol = [Vector(v) for v in collect(PolarDirections(100))]
Rpol = overapproximate(R, dirs_pol);

plot(set(R))
plot!(set(Rpol), alpha=.3)

In [ ]:
function requirement_1(sol::Vector{<:Flowpipe{N}}) where {N}

    Dx⁺ = [0, 1.0, 0, 0]
    Dx⁻ = [0, -1.0, 0, 0]
    dirs = [Dx⁺, Dx⁻]

    vtempl = Vector{TemplateReachSet{N, Vector{N}}}()
    sizehint!(vtempl, length(sol) * length(sol[1]))
    
    for fp in sol
        for X in fp
            Rtemplate = overapproximate(X, dirs)
            push!(vtempl, Rtemplate)
        end
    end
    return Flowpipe(vtempl)
end

In [ ]:
@time r1 = requirement_1(sol);

In [ ]:
typeof(t1)

$|x(t) - x_0|$ vale $x(t) - x_0$ si $x(t) \geq x_0$ o $x_0 - x(t)$ si $x(t) < x_0$.

In [ ]:
tend(r1)

In [ ]:
const IA = IntervalArithmetic

t1 = 98e-3 #23e-3
tf = tend(r1)
y = r1(IA.Interval(t1, tf))

max_D⁺ = maximum(sup_func(X, 1) for X in y)
@show max_D⁺ 
@show max_D⁺ < x0 + ε

max_D⁻ = maximum(sup_func(X, 2) for X in y)
@show max_D⁻
@show max_D⁻ < -x0 + ε

In [ ]:
-x0 + ε

In [ ]:
length(sol[1])

In [ ]:
plot([project(X, [0, 2]) for X in sol[600][1:100:end]], vars=(0, 2))
plot!(x -> x0 + ε, color=:red)
plot!(x -> x0 - ε, color=:red)

In [ ]:
tspan(sol[900])

In [ ]:
x0 - maximum(sup_func(X, 2) for X in y)

In [ ]:
[Interval(tspan(X)) × Interval(sup_func(X, 1), sup_func(X, 1)) for X in r1];

In [ ]:
using Plots

In [ ]:
plot([Interval(tspan(X)) × Singleton([sup_func(X, 1)]) for X in r1])

In [ ]:
plot!([Interval(tspan(X)) × Singleton([-sup_func(X, 2)]) for X in r1])

In [ ]:
# project(sol[1], [2]) # hacer
# project(sol[1][1], (2)) # HACER

intervalo_test = Interval(x0 - ε, x0 + ε)

X = [[set(project(X, [2])) for X in sol_i] for sol_i in sol];

In [ ]:
X[2][10] ⊆ intervalo_test

### Requirement 2

In [ ]:
# assumes that v = K/(i*drot) * I and
# I is the first (1) variable in the flowpipe
function velocity(sol, K, i, drot)
    M = hcat(K/(i*drot))
    if isa(M, Interval)
        M = IntervalMatrix(M)
    end

    V_fp = Vector{SparseReachSet}()
    for fp in sol
        for X in fp
            I_Δt = overapproximate(M * set(project(X, [1])), Zonotope)
            Δt = tspan(X)
            push!(V_fp, SparseReachSet(I_Δt, Δt, [1]))
        end
    end

    return V_fp
end

function requirement_2(sol::Vector{<:Flowpipe{N}}) where {N}

    Dx⁺ = [0, 1.0, 0, 0]
    Dx⁻ = [0, -1.0, 0, 0]
    dirs = [Dx⁺, Dx⁻]

    vtempl = Vector{TemplateReachSet{N, Vector{N}}}()
    sizehint!(vtempl, length(sol) * length(sol[1]))
    
    for fp in sol
        for X in fp
            sfunc = [ρ(Dx⁺, X), ρ(Dx⁻, X)]
            Δt = tspan(X)
            R = TemplateReachSet(dirs, sfunc, Δt)
            push!(vtempl, R)
        end
    end
    return vtempl
end